In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import json
import geopandas as gpd
import folium
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)
from uszipcode import SearchEngine
import plotly.graph_objects as go

In [180]:
# resids_df=pd.read_csv('../../data/sarimax_resids_acs_bikeshare.csv', 
#                       dtype={'Unnamed: 0': str}).rename(columns = {"Unnamed: 0":"zip"})

XGB_resids_auto_only_df=pd.read_csv('./../XGBoost_residuals/residuals_XGB-zri_trends_minmax.csv', 
                      dtype={'zip': str})

In [181]:
zipcodes = XGB_resids_auto_only_df['zip'].unique()

In [182]:
zips_geo_df = pd.read_csv('./../data/georef-united-states-of-america-zc-point.csv',
                          sep=';', dtype={'Zip Code': str})

In [183]:
zips_geo_df['Zip Code'] = zips_geo_df['Zip Code'].map(lambda x: '0' + x if len(x) < 5 else x)

In [184]:
zips_geo_df = zips_geo_df.loc[zips_geo_df['Zip Code'].isin(zipcodes)]
zips_geo_df['Zip Code'].nunique()

1100

In [185]:
# zips_geo_df

In [186]:
# XGB_resids_auto_only_df=pd.read_csv('../../data/residuals_XGB-zri_minmax.csv', 
#                       dtype={'zip': str})

In [187]:
XGB_resids_auto_only_df['zip'] = XGB_resids_auto_only_df['zip'].map(lambda x: '0' + x if len(x) < 5 else x)

In [188]:
XGB_resids_auto_only_df['residual']=XGB_resids_auto_only_df['zri_predict']-XGB_resids_auto_only_df['zri_actual']
XGB_resids_auto_only_df['residual_squared']=XGB_resids_auto_only_df['residual'].map(lambda x: x**2)
rmse_table=XGB_resids_auto_only_df.groupby('zip')[['residual_squared']].agg(['sum','count'])
rmse_table['residual_avgd']=rmse_table['residual_squared']['sum']/rmse_table['residual_squared']['count']
rmse_table['residual_avgd']=rmse_table['residual_avgd'].apply(lambda x: x**0.5)
rmse_XGB_resids_auto_only_df = rmse_table[['residual_avgd']].reset_index().rename(columns={'residual_avgd': "RMSE"})


In [189]:
rmse_XGB_resids_auto_only_df.columns = ['zip', 'rmse']

In [190]:
zip_county_merged = rmse_XGB_resids_auto_only_df.merge(zips_geo_df,
                                                       how= 'left', 
                                                       left_on= "zip", 
                                                       right_on = "Zip Code")

In [191]:
# zip_county_merged

In [192]:
# 

bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 100000)])
zip_county_merged['buckets'] = pd.cut(zip_county_merged['Density'], bins=[0, 500, 1000, 1000000], labels=['Rural', 'Suburban', 'Urban'])

In [193]:
# zip_county_merged

In [194]:
zip_buckets = zip_county_merged.groupby('buckets').agg(mean = ('rmse', 'mean'))

In [195]:
zip_buckets

,mean
buckets,
Rural,43.786960
Suburban,42.525020
Urban,53.900624


In [196]:
#zip_buckets_conc=zip_buckets.copy()
#zip_buckets_conc=pd.concat([zip_buckets_conc, zip_buckets],axis=1)
zip_buckets_conc

,mean,mean,mean,mean,mean
buckets,,,,,
Rural,44.039019,42.021597,43.926033,44.291568,43.786960
Suburban,44.028641,40.377465,43.774248,43.441534,42.525020
Urban,55.316444,51.752880,55.185102,53.626043,53.900624


In [197]:
zip_buckets_conc2=zip_buckets_conc.copy()

In [200]:
zip_buckets_conc2.columns = ['AR', 'ACS', 'Bikeshare', 'Econ', 'Trends']

In [205]:
ar = zip_buckets_conc2.loc[:,'AR']
acs = zip_buckets_conc2.loc[:,'ACS']
bikeshare = zip_buckets_conc2.loc[:,'Bikeshare']
econ = zip_buckets_conc2.loc[:,'Econ']
trends = zip_buckets_conc2.loc[:,'Trends']


In [232]:
all_ = pd.concat([ar, acs, bikeshare, econ, trends], axis = 0)

In [233]:
all_ = pd.DataFrame(all_)

In [234]:
all_['source'] = ['AR','AR','AR','ACS','ACS','ACS','Bikeshare','Bikeshare','Bikeshare','Econ','Econ','Econ','Trends','Trends','Trends']

In [235]:
all_ = all_.rename(columns = {0:'rmse'})

In [236]:
all_.loc[all_.index == 'Rural', 'rmse'] = all_.loc[all_.index == 'Rural', 'rmse']/44.039019

In [237]:
all_.loc[all_.index == 'Suburban', 'rmse'] = all_.loc[all_.index == 'Suburban', 'rmse']/44.028641

In [238]:
all_.loc[all_.index == 'Urban', 'rmse'] = all_.loc[all_.index == 'Urban', 'rmse']/55.316444

In [244]:
import plotly.express as px
fig = px.bar(all_, x=all_.index, y = all_['rmse'], color = all_['source'], barmode = 'group')
fig.update_yaxes(title = 'relative rmse')
fig.update_xaxes(title = 'population density')
fig.show()

In [229]:
all_

,rmse,source
buckets,,
Rural,1.000000,AR
Suburban,NaN,AR
Urban,1.000000,AR
Rural,0.954190,ACS
Suburban,NaN,ACS
Urban,0.935579,ACS
Rural,0.997434,Bikeshare
Suburban,NaN,Bikeshare
Urban,0.997626,Bikeshare
